In [1]:
from sklearn.datasets import load_digits
import numpy as np
from pylab import *
from PIL import Image
from PIL import ImageDraw
import numpy as np
from matplotlib import cm
import cv2

Кластеризация с помощью дескриптора

In [2]:
digits = load_digits()
deskriptors = []
for ind in range(len(digits.images)):
  img = digits.images[ind]
  sobelx = np.array([[-1,0,1], [-2,0,2], [-1,0,1]])
  sobely = np.array([[-1,-2,-1], [0,0,0], [1,2,1]])
  #градиенты
  imgx = cv2.filter2D(img, -1, sobelx)
  imgy = cv2.filter2D(img, -1, sobely)
  k = 0.06
  #Гаусс
  sigma = 1.3
  filter = np.zeros((3,3), dtype=np.float)
  p=1 #кол-во нулей
  for i in range(-1,2):
    for j in range(-1,2):
      filter[i+p,j+p]=np.exp(-(i**2+j**2)/(2*(sigma**2)))
  filter/=(np.sqrt(np.pi*2)*(sigma**2))
  filter/=filter.sum()

  #ищем R для почти всех точек
  array_R = []
  for a in range(6):
    for b in range(6):
      x = a
      y = b
      M = np.zeros((2,2))
      for u in range(filter.shape[0]):
        for v in range(filter.shape[1]):
          A = np.array([[imgx[x + u, y + v]**2, imgx[x + u, y + v] * imgy[x + u, y + v]], 
                        [imgy[x + u, y + v] * imgx[x + u, y + v], imgy[x + u, y + v]**2]])
          M += filter[u,v] * A
      if np.linalg.det(M) - k * np.trace(M)**2 > 0:
        array_R.append([x, y, np.linalg.det(M) - k * np.trace(M)**2])

  #фильтруем 
  array_R2 = []
  for t in range(len(array_R)):
    if array_R[t][2] > 400000:
      array_R2.append(array_R[t])

  #если их количество больше четырех
  while(len(array_R2)>4):
    max = 0
    max_ind = 0
    for t in range(len(array_R2)):
      if array_R2[t][2] > max:
        max = array_R2[t][2]
        max_ind = t
    array_R2.pop(max_ind)
  #паттерн пар точек
  import random
  S = [[[ 1,  1],
          [ 0,  1],
          [ 0,  0],
          [ 0, -1]],
        [[ 1,  0],
          [-1,  1],
          [ 1, -1],
          [ 0,  0]]]
  img2 = img.copy()
  for i in range(img.shape[0]-2):
    for j in range(img.shape[1]-2):
      img2[1+i,1+j]=np.sum(filter * img[i:i+3, j:j+3])
  img2 = img2.astype(np.uint8)
  desk_res = []
  for i in range(len(array_R2)):
    x0 = array_R2[i][0]
    y0 = array_R2[i][1]
    desk = np.zeros(4)
    for j in range(4):
      x1 = int(S[0][j][0]) + x0
      y1 = int(S[0][j][1]) + y0
      x2 = int(S[1][j][0]) + x0
      y2 = int(S[1][j][1]) + y0
      if x1>=0 and y1>=0 and x2>=0 and y2>=0 and x1<img2.shape[0] and x2<img2.shape[0] and y1<img2.shape[1] and y2<img2.shape[1]:
        if img2[x1, y1] < img2[x2, y2]:
          desk[j] = 1
    for k in range(4):
      desk_res.append(int(desk[k]))
  while len(desk_res) < 16:
    desk_res.append(0)
  deskriptors.append(desk_res)

len(deskriptors)

<ipython-input-2-db5622299476>:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  filter = np.zeros((3,3), dtype=np.float)


1797

In [3]:
images = digits.images#картинки
data = deskriptors#длина дескриптора - 16
k=10#количество кластеров
centroids = []#центроиды
for i in range(k):#инициализация центроидов
    centroids.append(data[i])#get_histogram(data[i]))#random.randint(0,100)])

In [4]:
def check(clasters, clasters_target):
    targets = [[] for x in range(k)]
    for i in range(len(clasters)):
        for j in range(len(clasters[i])):
            targets[i].append(digits.target[clasters_target[i][j]])
    true_count = 0
    count = 0
    for i in range(len(targets)):
        for j in range(len(targets[i])):
            count += 1
            if i == targets[i][j]:
                true_count += 1
    print(true_count/count)
    
def distance(ar1, ar2):#евклидово расстояние между двумя векторами
    dist = 0
    for i in range(len(ar1)):
        dist += (ar1[i]-ar2[i])**2
    return dist**0.5

return_centroids = []
const_centroids = False
while const_centroids == False:#цикл пока центроиды не перестанут меняться
    new_centroids_count = 0#количество центроидов, которые поменялись
    clasters = [[] for x in range(k)]#массив из 10 массивов, записыаем туда вектора
    clasters_target = [[] for x in range(k)]#массив из 10 массивов, записыаем туда цифру, которую нашли на картинке
    for i in range(len(images)):#для каждой картинки находим минимальное расстояние до центроида
        dist_array = []#записываем расстояние до разных центроидов
        for j in range(k):
            dist = distance(data[i], centroids[j])
            dist_array.append(dist)
        index = dist_array.index(min(dist_array))#индекс минимального расстояния
        clasters[index].append(data[i])
        clasters_target[index].append(i)
    for i in range(k):#пересчитываем центроиды
        sum_array = [0] * 16
        for j in range(len(clasters[i])):#считаем сумму всех векторов одного кластера
            for t in range(len(sum_array)):
                sum_array[t] += clasters[i][j][t]
        new_centroid = []
        for d in range(16):
            if sum_array[d]==0:
                new_centroid.append(0)
            else:
                num = sum_array[d]/len(clasters[i])
                if num < 0.5:
                    new_centroid.append(0)
                else:
                    new_centroid.append(1)
        if list(centroids[i]) != new_centroid:#если центроид изменился
            new_centroids_count += 1
            centroids[i] = new_centroid
    if new_centroids_count == 0:#если все центроиды остались такими же, выходим из цикла
        const_centroids = True
        return_centroids = centroids
    check(clasters, clasters_target)

0.1352253756260434
0.13689482470784642
0.12409571508069003
0.11686143572621036


In [5]:
matrix = np.zeros((10,10), dtype=int32)
targets = [[] for x in range(k)]
for i in range(len(clasters)):
    for j in range(len(clasters[i])):
        targets[i].append(digits.target[clasters_target[i][j]])
#print(targets)
true_count = 0
count = 0
for i in range(len(targets)):
    for j in range(len(targets[i])):
        count += 1
        matrix[i, targets[i][j]]+= 1
print(matrix)

[[ 48   4  19  34  31  46  51  29   2  24]
 [ 26  12  57  72   6  55  37  91   2  59]
 [ 21   1  23   3   7  17  18  18   3   0]
 [ 77 117  57  69  69  31  71  30 164  69]
 [  5  48  21   5  47  22   2   9   1  27]
 [  1   0   0   0  21  11   2   2   2   1]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]


Кластеризация по картинке

In [14]:
digits = load_digits()
images = digits.images
data = digits.data
k=10 #количество кластеров
centroids = [] #центроиды
for i in range(k): #инициализация центроидов
    centroids.append(data[i])

In [15]:
def distance(ar1, ar2):#евклидово расстояние между двумя векторами
    dist = 0
    for i in range(len(ar1)):
        dist += (ar1[i]-ar2[i])**2
    return dist**0.5

return_centroids = []
const_centroids = False
while const_centroids == False:#цикл пока центроиды не перестанут меняться
    new_centroids_count = 0#количество центроидов, которые поменялись
    clasters = [[] for x in range(k)]#массив из 10 массивов, записыаем туда вектора
    clasters_target = [[] for x in range(k)]#массив из 10 массивов, записыаем туда цифру, которую нашли на картинке
    for i in range(len(images)):#для каждой картинки находим минимальное расстояние до центроида
        dist_array = []#записываем расстояние до разных центроидов
        for j in range(k):
            dist = distance(data[i], centroids[j])
            dist_array.append(dist)
        index = dist_array.index(min(dist_array))#индекс минимального расстояния
        clasters[index].append(data[i])
        clasters_target[index].append(i)
    for i in range(k):#пересчитываем центроиды
        sum_array = [0] * 64
        for j in range(len(clasters[i])):#считаем сумму всех векторов одного кластера
            for t in range(len(sum_array)):
                sum_array[t] += clasters[i][j][t]
        new_centroid = [c / len(clasters[i]) for c in sum_array]#делим на количество элементов в кластере, это будет новый центроид
        if list(centroids[i]) != new_centroid:#если центроид изменился
            new_centroids_count += 1
            centroids[i] = new_centroid
    if new_centroids_count == 0:#если все центроиды остались такими же, выходим из цикла
        const_centroids = True
        return_centroids = centroids
    
    #####
    targets = [[] for x in range(k)]
    for i in range(len(clasters)):
        for j in range(len(clasters[i])):
            targets[i].append(digits.target[clasters_target[i][j]])
    true_count = 0
    count = 0
    for i in range(len(targets)):
        for j in range(len(targets[i])):
            count += 1
            if i == targets[i][j]:
                true_count += 1
    print(true_count/count)

0.5982192543127435
0.6410684474123539
0.6282693377851976
0.6021146355036171
0.5798553144129104
0.5470228158041179
0.5258764607679466
0.5281023928770172
0.5325542570951586
0.5358931552587646
0.535336672231497
0.5364496382860323
0.5381190873678353
0.5381190873678353


In [16]:
matrix = np.zeros((10,10), dtype=int32)
targets = [[] for x in range(k)]
for i in range(len(clasters)):
    for j in range(len(clasters[i])):
        targets[i].append(digits.target[clasters_target[i][j]])
#print(targets)
true_count = 0
count = 0
for i in range(len(targets)):
    for j in range(len(targets[i])):
        count += 1
        matrix[i, targets[i][j]]+= 1
print(count)
print(matrix)

1797
[[177   0   1   0   0   0   1   0   0   0]
 [  0 100   2   0   5   0   3   0  10   0]
 [  0  54   2   0   5   0   0   2   6  20]
 [  0  25 148   2   0   0   0   0   3   0]
 [  1   0   0   0 161   1   0   0   0   0]
 [  0   0  12 161   0  39   0   0  13 145]
 [  0   2   0   0   0   2 176   0   1   0]
 [  0   0   2   8   8   0   0 170   3   8]
 [  0   0  10  11   2   0   1   3 135   2]
 [  0   1   0   1   0 140   0   4   3   5]]


Кластеризация по картинке

In [44]:
def hist(data):
    result = []
    for i in range(len(data)):
      img = data[i]
      vect = np.zeros(17, dtype=int64)
      for j in range(len(img)):
        vect[int(img[j])] += 1
      result.append(vect)
    return result


digits = load_digits()
images = digits.images
data = hist(digits.data)
print(data)
k=10#количество кластеров
centroids = []#центроиды
for i in range(k):#инициализация центроидов
    centroids.append(data[i])#data[i])#get_histogram(data[i]))#random.randint(0,100)])

[array([29,  2,  2,  1,  2,  4,  1,  1,  5,  2,  3,  2,  3,  3,  1,  3,  0]), array([34,  3,  1,  2,  0,  1,  3,  1,  0,  1,  1,  2,  1,  1,  0,  2, 11]), array([30,  3,  0,  3,  1,  2,  1,  0,  3,  2,  0,  3,  1,  3,  1,  4,  7]), array([31,  6,  2,  0,  2,  1,  1,  2,  3,  2,  1,  1,  2,  6,  1,  3,  0]), array([34,  3,  3,  1,  2,  1,  2,  2,  2,  1,  2,  1,  0,  2,  0,  3,  5]), array([33,  1,  0,  0,  4,  1,  0,  3,  0,  2,  3,  1,  2,  1,  2,  1, 10]), array([35,  1,  1,  3,  0,  1,  0,  2,  2,  1,  0,  2,  2,  5,  1,  3,  5]), array([32,  3,  1,  0,  3,  2,  1,  3,  4,  1,  0,  2,  1,  3,  0,  6,  2]), array([26,  4,  3,  3,  2,  0,  0,  0,  3,  2,  2,  2,  3,  1,  4,  3,  6]), array([32,  2,  2,  3,  1,  0,  0,  0,  0,  4,  1,  2,  4,  3,  1,  3,  6]), array([26,  5,  1,  2,  3,  1,  1,  0,  5,  3,  4,  3,  2,  1,  1,  1,  5]), array([34,  4,  1,  1,  1,  2,  0,  0,  0,  1,  1,  0,  3,  2,  4,  1,  9]), array([35,  3,  3,  1,  1,  1,  1,  1,  2,  2,  1,  1,  3,  2,  2,  3,  2])

In [46]:
def distance(ar1, ar2):#евклидово расстояние между двумя векторами
    dist = 0
    for i in range(len(ar1)):
        dist += (ar1[i]-ar2[i])**2
    return dist**0.5

return_centroids = []
const_centroids = False
while const_centroids == False:#цикл пока центроиды не перестанут меняться
    new_centroids_count = 0#количество центроидов, которые поменялись
    clasters = [[] for x in range(k)]#массив из 10 массивов, записыаем туда вектора
    clasters_target = [[] for x in range(k)]#массив из 10 массивов, записыаем туда цифру, которую нашли на картинке
    for i in range(len(images)):#для каждой картинки находим минимальное расстояние до центроида
        dist_array = []#записываем расстояние до разных центроидов
        for j in range(k):
            dist = distance(data[i], centroids[j])
            dist_array.append(dist)
        index = dist_array.index(min(dist_array))#индекс минимального расстояния
        clasters[index].append(data[i])
        clasters_target[index].append(i)
    for i in range(k):#пересчитываем центроиды
        sum_array = [0] * 17
        for j in range(len(clasters[i])):#считаем сумму всех векторов одного кластера
            for t in range(len(sum_array)):
                sum_array[t] += clasters[i][j][t]
        new_centroid = [c / len(clasters[i]) for c in sum_array]#делим на количество элементов в кластере, это будет новый центроид
        if list(centroids[i]) != new_centroid:#если центроид изменился
            new_centroids_count += 1
            centroids[i] = new_centroid
    if new_centroids_count == 0:#если все центроиды остались такими же, выходим из цикла
        const_centroids = True
        return_centroids = centroids
    
    #####
    targets = [[] for x in range(k)]
    for i in range(len(clasters)):
        for j in range(len(clasters[i])):
            targets[i].append(digits.target[clasters_target[i][j]])
    true_count = 0
    count = 0
    for i in range(len(targets)):
        for j in range(len(targets[i])):
            count += 1
            if i == targets[i][j]:
                true_count += 1
    print(true_count/count)

0.21146355036171396
0.2109070673344463
0.2020033388981636
0.20645520311630494
0.21146355036171396
0.21202003338898165
0.21146355036171396
0.20812465219810797
0.21202003338898165
0.21368948247078465
0.2142459654980523
0.21480244852532
0.2142459654980523
0.2142459654980523
0.21368948247078465
0.2142459654980523
0.2142459654980523
0.2142459654980523
0.2142459654980523
0.21368948247078465
0.21368948247078465


In [47]:
matrix = np.zeros((10,10), dtype=int32)
targets = [[] for x in range(k)]
for i in range(len(clasters)):
    for j in range(len(clasters[i])):
        targets[i].append(digits.target[clasters_target[i][j]])
#print(targets)
true_count = 0
count = 0
for i in range(len(targets)):
    for j in range(len(targets[i])):
        count += 1
        matrix[i, targets[i][j]]+= 1
print(count)
print(matrix)

1797
[[72  0  1  5  2 21  0  2 16 21]
 [ 0 92  8 11 13 20  6 13  1  2]
 [10  4 51 37 48 33 36 26 25 22]
 [18  0  7 18  4 13 23  9 22 28]
 [ 1  7 34 25 44 27 27 39 11 16]
 [ 7 36 11  8 14 14 20 13  9 12]
 [ 1 43 13 11 17 16 13 32  5  7]
 [ 9  0 22 31 15 17 30 19 22 23]
 [49  0 13 18  9  9  8  9 43 31]
 [11  0 17 19 15 12 18 17 20 18]]
